In [1]:
import numpy as np
import pandas as pd
import requests

In [2]:
n_days = 1
query = 'binderhub_launch_time_seconds_count{}{}'
query_days = "[{}d]".format(n_days)
query_selectors = "{status='success'}"
query = query.format(query_selectors, query_days)
resp = requests.get('https://notebooks.gesis.org/prometheus/api/v1/query', params={'query': query})
data = resp.json()['data']['result']

In [3]:
df = {'name': [], 'org': [], 'provider': [], 'builds': []}
for container in data:
    if 'repo' not in container['metric']:
        continue
    pref, org, repo = container['metric']['repo'].rsplit('/', 2)
    provider = pref.replace('https://', '')
    counts = [int(ii[1]) for ii in container['values']]
    builds = np.max(counts)
    df['name'].append(repo)
    df['org'].append(org)
    df['provider'].append(provider)
    df['builds'].append(builds)
df = pd.DataFrame(df)
df = df.drop_duplicates(['name'])
df = df.groupby(['name', 'org', 'provider']).sum().reset_index().sort_values('builds', ascending=False)
df['log_builds'] = df['builds'].apply(np.log)